In [124]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

In [3]:
#helper functions

def read_file(filename, pit_state=False):
    change = []
    data = []
    for i in range(len(filename.sheet_names)):
        if pit_state==True:
            if i==0:
                change = pd.read_excel(filename, 0)
            else:
                data.append(pd.read_excel(filename, i))
        else:
            data.append(pd.read_excel(filename,i))
    if pit_state==True: return change, data
    return data

## Raw Data

In [125]:
#PIT State Data
xls = pd.ExcelFile('Data/2007-2022-PIT-Counts-by-State.xlsx')
PIT_State_Change, PIT_State = read_file(xls, True)

xls = pd.ExcelFile('Data/2011-2022-PIT-Veteran-Counts-by-State.xlsx')
PIT_Veteran_State_Change, PIT_Veteran_State = read_file(xls, True)

#PIT CoC Data
xls = pd.ExcelFile('Data/2007-2022-PIT-Counts-by-CoC.xlsx')
PIT_CoC = read_file(xls)

xls = pd.ExcelFile('Data/2011-2022-PIT-Veteran-Counts-by-CoC.xlsx')
Veteran_CoC = read_file(xls)

#HIC State Data
xls = pd.ExcelFile('Data/2007-2022-HIC-Counts-by-State.xlsx')
HIC_State = read_file(xls)

#HIC CoC Data
xls = pd.ExcelFile('Data/2007-2022-HIC-Counts-by-CoC.xlsx')
HIC_CoC = read_file(xls)

#Funding Data
xls = pd.ExcelFile('Data/All-States-2020-2021-Funding-Report.xlsx')
Funding = read_file(xls)

## Data Cleaning

In [275]:
#PIT_State_Change.head()
#AS: American Samoa, no data?
#MP: Northern Mariana Islands, PR: Puerto Rico, TT: Trust Territories

#get rid of states/territories that have no data for total change
copy = PIT_State_Change.copy()
copy = copy.replace(' ', float('NaN'), regex = True)
copy.dropna(subset = ['Change in Total Homelessness, ' + str(2022-(i+1)) + '-' + '2022'], inplace=True)
PIT_State_Change_vF = copy

In [276]:
#PIT_State_Data[0].head()
#PIT_State_Data[-1].head() --> 2007 has no data? maybe we can extrapolate

#get rid of states/territories that did not collect any homeless data
PIT_State_vF = []
for i in range(len(PIT_State) - 1):
    copy = PIT_State[i].copy()
    copy = copy.replace(' ', float('NaN'), regex = True)
    copy.dropna(subset = ['Overall Homeless, ' + str(2022-i)], inplace=True)
    PIT_State_vF.append(copy)

In [277]:
#PIT_CoC_Data[0]
#PIT_CoC_Data[-1]
#PIT_CoC_Data[-2]


#get rid of any CoCs that did not collect any homeless data
PIT_CoC_vF = []
for i in range(len(PIT_CoC) - 2):
    copy = PIT_CoC[i].copy()
    copy = copy.replace(' ', float('NaN'), regex = True)
    copy.dropna(subset = ['Overall Homeless, ' + str(2022-i)], inplace=True)
    PIT_CoC_vF.append(copy)

In [278]:
#Funding[0]

#remove unnecessary columns
Funding_vF = []
for i in range(len(Funding)):
    copy = Funding[i].copy()
    if i==0:
        Funding_vF.append(copy.drop(columns=['CoC Name', 'Organization Name', 'Grant Number']))
    else:
        Funding_vF.append(copy.drop(columns=['CoC Name', 'Organization Name']))

In [279]:
#PIT_Veteran_State[0]

#remove states that don't collect veteran data
PIT_Veteran_State_vF = []
for i in range(len(PIT_Veteran_State)):
    copy = PIT_Veteran_State[i].copy()
    copy = copy.replace(' ', float('NaN'), regex = True)
    copy.dropna(subset = ['Homeless Veterans, ' + str(2022-i)], inplace=True)
    PIT_Veteran_State_vF.append(copy)

In [280]:
#PIT_Veteran_Change
#AS seems to never track, let's delete

copy = PIT_Veteran_Change.copy()
PIT_Veteran_Change_vF = copy.drop([3])

In [281]:
#HIC_State[-1]

#get rid of unnecessary header, drop any states/territories that don't track total beds
HIC_State_vF = []
for i in range(len(HIC_State) - 1):
    copy = HIC_State[i].copy()
    copy = copy.rename(columns=copy.iloc[0]).drop(copy.index[0]).reset_index(drop=True)
    copy = copy.replace(' ', float('NaN'), regex = True)
    copy.dropna(subset = [copy.columns[1]], inplace=True)
    HIC_State_vF.append(copy)

In [282]:
#HIC_CoC[-1]

#get rid of unnecessary header, drop any states/territories that don't track total beds
HIC_CoC_vF = []
for i in range(len(HIC_CoC) - 1):
    copy = HIC_CoC[i].copy()
    copy = copy.rename(columns=copy.iloc[0]).drop(copy.index[0]).reset_index(drop=True)
    copy = copy.replace(' ', float('NaN'), regex = True)
    copy.dropna(subset = [copy.columns[1]], inplace=True)
    HIC_CoC_vF.append(copy)

## Export Cleaned Excel Files

In [291]:
PIT_State_Change_vF.to_excel(r'Cleaned Data/PIT_State_Change_vF.xlsx',index=False)

In [ ]:
with pd.ExcelWriter(r'Cleaned Data/PIT_State_vF.xlsx') as writer:
    for i in range(len(PIT_State_vF)):
        PIT_State_vF[i].to_excel(writer, sheet_name=str(2022-i), index=False)

In [ ]:
with pd.ExcelWriter(r'Cleaned Data/PIT_CoC_vF.xlsx') as writer:
    for i in range(len(PIT_CoC_vF)):
        PIT_CoC_vF[i].to_excel(writer, sheet_name=str(2022-i), index=False)

In [294]:
with pd.ExcelWriter(r'Cleaned Data/Funding_vF.xlsx') as writer:
    for i in range(len(Funding_vF)):
        Funding_vF[i].to_excel(writer, sheet_name=str(2020+i), index=False)

In [296]:
with pd.ExcelWriter(r'Cleaned Data/PIT_Veteran_State_vF.xlsx') as writer:
    for i in range(len(PIT_Veteran_State_vF)):
        PIT_Veteran_State_vF[i].to_excel(writer, sheet_name=str(2022-i), index=False)

In [297]:
PIT_Veteran_Change_vF.to_excel(r'Cleaned Data/PIT_Veteran_Change_vF.xlsx',index=False)

In [300]:
with pd.ExcelWriter(r'Cleaned Data/HIC_State_vF.xlsx') as writer:
    for i in range(len(HIC_State_vF)):
        HIC_State_vF[i].to_excel(writer, sheet_name=str(2022-i), index=False)

In [301]:
with pd.ExcelWriter(r'Cleaned Data/HIC_CoC_vF.xlsx') as writer:
    for i in range(len(HIC_CoC_vF)):
        HIC_CoC_vF[i].to_excel(writer, sheet_name=str(2022-i), index=False)